the ipynb is refer: 
Create the premodel by ResNet50,Xception, InceptionV3, VGG16, VGG19

In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle

np.random.seed(2017)

def one_hot_encode(y):
    l = list()
    for item in y:
        c = [0. for i in range(120)]
        c[item] = 1.
        l.append(c)
    return np.array(l)

def load_data(pretrain_data_file):
    print('load data: ' + pretrain_data_file)
    with h5py.File("models/premodel/" + pretrain_data_file, 'r') as h:
        X_train = np.array(h['train'])
        X_test = np.array(h['test'])
        y_train = np.array(h['label'])
        y_train = one_hot_encode(y_train)
        X_train, y_train = shuffle(X_train, y_train)
        return X_train, y_train, X_test
    

from keras.models import *
from keras.layers import *
from keras.callbacks import *

def make_model(input_shape):
    input_tensor = Input(input_shape)
    x = Dropout(0.5)(input_tensor)
    x = Dense(120, activation='sigmoid')(x)
    model = Model(input_tensor, x)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model
print("Done")

Using TensorFlow backend.


Done


In [2]:
earlystopping = EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')

ResNet50

In [3]:
X_train, y_train, X_test = load_data("gap_ResNet50.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128,  epochs=100, validation_split=0.05, callbacks=[earlystopping] )

load data: gap_ResNet50.h5
Train on 19421 samples, validate on 1023 samples
Epoch 1/100
19421/19421 [==============================] - 0s - loss: 3.1678 - acc: 0.2531 - val_loss: 1.6235 - val_acc: 0.5640
Epoch 2/100
19421/19421 [==============================] - 0s - loss: 1.6812 - acc: 0.5312 - val_loss: 1.2994 - val_acc: 0.6461

Xception

In [4]:
X_train, y_train, X_test = load_data("gap_Xception.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_split=0.05, callbacks=[earlystopping])

load data: gap_Xception.h5
Train on 19421 samples, validate on 1023 samples
Epoch 1/100
19421/19421 [==============================] - 0s - loss: 1.6086 - acc: 0.7786 - val_loss: 0.4512 - val_acc: 0.9120
Epoch 2/100
19421/19421 [==============================] - 0s - loss: 0.3633 - acc: 0.9140 - val_loss: 0.3133 - val_acc: 0.9179

InceptionV3

In [5]:
X_train, y_train, X_test = load_data("gap_InceptionV3.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_split=0.05, callbacks=[earlystopping])

load data: gap_InceptionV3.h5
Train on 19421 samples, validate on 1023 samples
Epoch 1/100
19421/19421 [==============================] - 0s - loss: 1.3824 - acc: 0.7360 - val_loss: 0.3694 - val_acc: 0.9032
Epoch 2/100
19421/19421 [==============================] - 0s - loss: 0.3295 - acc: 0.9041 - val_loss: 0.2890 - val_acc: 0.9150

VGG16

In [6]:
X_train, y_train, X_test = load_data("gap_VGG16.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_split=0.05, callbacks=[earlystopping])

load data: gap_VGG16.h5
Train on 19421 samples, validate on 1023 samples
Epoch 1/100
19421/19421 [==============================] - 0s - loss: 5.7309 - acc: 0.0097 - val_loss: 4.7364 - val_acc: 0.0068
Epoch 2/100
19421/19421 [==============================] - 0s - loss: 4.7408 - acc: 0.0105 - val_loss: 4.6433 - val_acc: 0.0039

VGG19

In [7]:
X_train, y_train, X_test = load_data("gap_VGG19.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_split=0.05, callbacks=[earlystopping])

load data: gap_VGG19.h5
Train on 19421 samples, validate on 1023 samples
Epoch 1/100
19421/19421 [==============================] - 0s - loss: 5.5429 - acc: 0.0123 - val_loss: 4.7042 - val_acc: 0.0078
Epoch 2/100
19421/19421 [==============================] - ETA: 0s - loss: 4.6907 - acc: 0.0153

combine  ResNet50,Xception and InceptionV3

In [8]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in [
#    "gap_ResNet50.h5", 
    "gap_Xception.h5", 
    "gap_InceptionV3.h5",
#     "gap_VGG16",
#     "gap_VGG19"
    ]:
    print('------------------'+filename)
    with h5py.File("models/premodel/" + filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)
y_train = one_hot_encode(y_train)

X_train, y_train = shuffle(X_train, y_train)
print(X_train.shape)
print(y_train.shape)

------------------gap_Xception.h5
------------------gap_InceptionV3.h5
(20444, 4096)
(20444, 120)


In [9]:
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_split=0.05, callbacks=[earlystopping])

Train on 19421 samples, validate on 1023 samples
Epoch 1/100
19421/19421 [==============================] - 0s - loss: 0.9031 - acc: 0.8197 - val_loss: 0.2583 - val_acc: 0.9316
Epoch 2/100
19421/19421 [==============================] - 0s - loss: 0.2133 - acc: 0.9357 - val_loss: 0.2

In [30]:
y_pred = model.predict(X_test, verbose=1)

import pandas as pd
from keras.preprocessing.image import *

dir = "/ext/Data/Dog_Breed_Identification/Kaggle"
dog_imgs_list_csv = os.path.join(dir, "labels.csv")
df = pd.read_csv(dog_imgs_list_csv)
#driver_list = df.groupby('subject', as_index=False)['img'].count()
breed_list = df.groupby('breed')['id'].count()
classes = list()
for index, value in breed_list.iteritems():
    classes.append(index)
    
image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(os.path.join(dir, "test"), image_size, shuffle=False, 
                                         batch_size=128, class_mode=None)

l = list()
for i, fname in enumerate(test_generator.filenames):
    name = fname[fname.rfind('/')+1:fname.rfind('.')]
    l.append([name, *y_pred[i]])

l = np.array(l)
data = {'id': l[:,0]}
for i in range(120):
    data[classes[i]] = l[:,i+1]
df = pd.DataFrame(data, columns=['id'] +  classes)

df.to_csv('pred/pred.csv', index=None)
df.head(10)

10304/10357 [============================>.] - ETA: 0sFound 10357 images belonging to 1 classes.


,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,13b4953cc9afdcf603692f74a7cf8048,4.27443e-07,6.00847e-07,0.631334,1.4963e-07,2.13137e-06,5.21648e-07,2.2289e-08,1.13076e-06,9.19218e-08,...,9.20945e-07,2.94106e-07,5.17308e-06,7.33559e-07,6.53259e-07,2.87571e-07,3.42536e-07,3.37112e-06,2.78023e-08,8.78578e-07
1,fcba7f675d9fcacd449174f2229ca33b,1.87925e-10,1.59403e-10,3.34613e-10,3.91796e-08,1.21215e-09,9.94387e-09,3.55904e-09,6.14785e-09,4.2532e-10,...,6.10503e-11,1.73705e-08,1.41949e-09,1.73078e-08,8.94755e-11,9.31482e-10,2.27405e-09,2.90739e-09,0.00576781,1.43808e-10
2,8bf8defe18c6db0efbd60d80706b1eab,1.00651e-08,3.09341e-07,2.48548e-07,1.21473e-06,3.18734e-08,8.94765e-06,4.68611e-07,4.66775e-08,3.69758e-08,...,1.73364e-08,5.3601e-08,8.85734e-09,2.63466e-07,6.63885e-08,3.70366e-08,6.72559e-09,2.72334e-08,2.32894e-08,1.59872e-07
3,5f0689f1454032a485c455101b4cf2ec,8.58625e-11,1.30312e-12,1.14028e-11,2.83921e-10,3.24579e-10,1.32823e-11,1.80059e-13,1.03413e-10,4.68509e-12,...,5.50386e-11,1.03556e-10,3.69406e-11,5.5776e-12,6.47311e-09,1.4396e-12,7.06664e-11,1.13717e-08,5.18973e-12,1.11503e-11
4,8b1ada6dd7072a18f649edbcfa01963c,4.96613e-07,3.24889e-08,1.68321e-06,5.15304e-08,1.82351e-06,9.25855e-08,3.87898e-08,1.7658e-06,2.28035e-08,...,4.88708e-08,6.75923e-05,3.03206e-08,2.88768e-07,1.70041e-07,8.49254e-08,4.97661e-07,3.04326e-07,3.62369e-08,1.21002e-07
5,d6937cacbbb5d64b535842861379f3b1,5.03035e-09,2.01785e-09,1.26545e-08,8.22541e-09,5.09003e-08,4.41124e-09,0.0239015,2.3916e-08,2.40867e-09,...,4.18482e-10,5.66526e-07,2.50769e-09,2.04826e-09,2.83968e-09,3.44239e-09,1.19412e-08,7.49601e-09,3.94658e-08,3.93621e-06
6,08f4ff17cc6bf754d798eb3b39fafebe,6.86878e-08,2.21539e-07,1.39546e-06,5.48124e-08,1.32723e-07,2.33662e-07,7.99135e-08,1.04144e-07,1.30126e-07,...,1.69801e-08,9.25018e-08,5.84695e-07,6.60531e-08,4.38403e-07,1.06785e-07,7.20435e-08,2.13295e-07,1.12587e-08,7.72291e-08
7,ca97512cc0f458942025ee480023bc95,2.06904e-06,4.42915e-07,1.19591e-06,1.02942e-06,4.02436e-07,4.76895e-07,2.48409e-07,1.83749e-07,7.77765e-07,...,2.48716e-06,7.23335e-07,7.12995e-07,1.99592e-06,3.5205e-06,6.5713e-07,6.03882e-07,7.41969e-07,1.50678e-08,8.40696e-07
8,7733032f8b54dfb3c173df864d48030a,6.74229e-08,4.2674e-08,5.57565e-08,7.57927e-08,5.48345e-06,1.3179e-06,2.12771e-07,2.03576e-07,1.44483e-07,...,1.58589e-08,2.54686e-07,2.10979e-07,1.64444e-06,2.63558e-07,2.79006e-07,1.33601e-06,3.22633e-07,3.84216e-07,1.78295e-07
9,c856e2561c1f0e9fcc96fcdce6b5cb10,1.16385e-09,7.86983e-11,4.36181e-10,2.45429e-09,5.37884e-10,5.15711e-09,8.82184e-11,1.89756e-09,2.12176e-09,...,4.32653e-11,5.71175e-09,8.48758e-10,0.0417012,3.07074e-09,2.97082e-10,2.76748e-09,3.60115e-09,5.91096e-09,5.61772e-10


In [31]:
#zip
import zipfile
f = zipfile.ZipFile('pred/pred.csv.zip', 'w' ,zipfile.ZIP_DEFLATED)
f.write('pred/pred.csv')
f.close()
print("create zip file done")

create zip file done
